# Statistics Coded: How do women and men use their time - statistics 

This noteboook reproduces step by step the figures avialable in the original [Statistics Explaind page](https://ec.europa.eu/eurostat/statistics-explained/index.php?title=How_do_women_and_men_use_their_time_-_statistics) and in the [source excel file](https://ec.europa.eu/eurostat/statistics-explained/images/2/2e/SE_Article_Charts_HETUS_2010_FINAL_06.09.2019.xlsx).

## Figure 1: Mean time spent on daily activities, all individuals by country, (hh:mm; 2008 to 2015)


The data is in the *tus_00age* dataset. We use the [restatapi](https://github.com/eurostat/restatapi) package to download the data. 

In [1]:
library(restatapi)
dt<-get_eurostat_data("tus_00age",filters("sleeping"),label=T)
dt

The code failed because of a fatal error:
	Error sending http request and maximum retry encountered..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.
